# Object-oriented scientific programming with C++

Matthias Möller, Jonas Thies, Cálin Georgescu, Jingya Li (Numerical Analysis, DIAM)

Lecture 5

## <center>Goal of this lecture</center>

<a href="#/3">Enumerators </a>

<a href="#/6">Type aliases </a>

<a href="#/10">Variadic template parameters</a>

<a href="#/17">C++ standard container classes and algorithms</a>

<a href="#/22">Iterators</a>

Range-based and for-each for loops

## <center>Enumerators</center>
Enumerators make it possible to collect named values
`enum Color { red, green, blue };`

Named values are mapped to, e.g., `red=0, green=1, blue=2`

Usage

```C++
Color col = Color::red; switch col {
      case Color::red:   // do something
          break;
      case Color::green: // do something else
          break;
}
```

## <center>Enumerators</center>
Enumerators can be initialised explicitly
```C++
enum Color { red=2, green=4, blue=8 };
```
Enumerators can be derived from a particular integral type
```C++
enum Color : int { red=2, green=4, blue=8 };
```
Enumerators can make use of arithmetic operations
```C++
enum Color { red=2, green=4, blue=8,
               cyan = red + green   };
```
However, an enumerator must not occur more than once

`enum TrafficLight {`<code>~~red~~</code>`, yellow,`<code>~~green~~</code>` };`

## <center>Scoped Enumerators</center>
C++11 introduces **scoped enumerators** which can occur
more than once (since they have different scopes!)
```C++
enum class Color { red, green, blue };
enum class TrafficLight { red, yellow, green };
```
For the rest, scoped enumerators can be used exactly in the same way as non-scoped enumerators

```C++
enum class Color { red=2, green=4, blue=8 };
enum class Color : int { red=2, green=4, blue=8 };
enum class Color { red=2, green=4, blue=8,
                   cyan = red + green   };
```

## <center>Type aliases</center>
_Implementation I_: type aliases via <span style=color:red;>typedef</span>
```C++
template<typename T, T v>
struct trait {
      typedef T type;           // type is a type
      static const T value = v; // value is a variable
  };
```

_Implementation II (since C++11)_: type aliases via <span style=color:red;>using</span>

```C++
template<typename T, T v>
struct trait {
      using type = T;           // type is a type
      static const T value = v; // value is a variable
  };
```

## <center>Type aliases</center>

https://www.online-ide.com/RLlfvTsGpK

In [ ]:
#include <iostream>
#include <typeinfo>


template<typename T, T v>
struct trait {
    typedef T type;           // type is a type
    static const T value = v; // value is a variable
};

int main() {
    typedef trait<int, 10> mytrait; // before C++11
    // using mytrait = trait<int, 10>; // since C++11

    std::cout << mytrait::value << " "
              << typeid(mytrait::type).name()
              << std::endl;

    return 0;
}

## <center>Intermezzo: `using` vs. `typedef`</center>
Remember the function pointers from session 3
```C++
const double myfunc1(double x) { return x; }
int main() {
      using funcPtr = double(*) (double);
      funcPtr f = myfunc1;
      std::cout << f(2.3) << std::endl;
}
```

## <center>Intermezzo: `using` vs. `typedef`</center>
This becomes much less intuitive with <span style=color:red;>typedef</span>

In [ ]:
#include <iostream>

double myfunc1(double x) {
    return x;
}

int main() {
    typedef double (funcPtr)(double); // Function pointer type
    funcPtr* f = myfunc1;               // Assign myfunc1 to pointer f
    std::cout << f(2.3) << std::endl;  // Use the function pointer to call myfunc1

    return 0;
}

## <center>Variadic templates</center>
**Task:** implement a function that takes an **arbitrary number** of possibly **different variables** and computes their sum
```C++
cout << sum(1.0)                                  << endl;
cout << sum(1.0, 1.0)                             << endl;
cout << sum(1.0, (int)1)                          << endl;
cout << sum(1.0, (int)1, (float)1.3, (double)1.3) << endl;
```

## <center>Variadic templates</center>
None of the template meta programming techniques we know so far will solve this problem with satisfaction

New concept in C++11: **variadic template parameters**

**Idea:** reformulate the problem as “one + rest”:

`sum(x1,x2,x3,...,xn) = x1 + sum(x2,x3,...,xn)`

That is, we combine recursion and function overloading with the ability to accept an arbitrary parameter list

## <center>Variadic templates</center>
Function overload for **one argument**
```C++
template<typename T>
double sum(T arg) { return arg; }
```
Function overload for **more than one argument**
```C++
template<typename T, typename ... Ts>
double sum(T arg, Ts ... args)
{ return arg + sum(args...); }
```
The **template parameter pack** 
```C++
template<typename ... Ts>
```
accepts zero or more template arguments but there can only be one template parameter pack per function.

## <center>Variadic templates</center>
The number of arguments in the parameter pack can be
detected using the `sizeof...()` function
```C++
template<typename ... Ts>
  int length(Ts ... args)
  {
      return sizeof...(args);
  }
```
**Task:** Write a type trait that determines the number of arguments passed to a function as parameter pack. In other words, implement the `sizeof...()` function yourself.

## <center>Automatic return type deduction</center>
**Task:** Implement the sum function for an arbitrary number of parameters using automatic return type deduction

Function overload for **one argument** (with C++11)
```C++
template<typename T>
auto sum(T arg) -> decltype(arg)
{ return arg; }
```

Function overload for **one argument** (with C++14)
```C++
template<typename T>
auto sum(T arg)
{ return arg; }
```

## <center>Automatic return type deduction</center>
Function overload for **more than one argument** (C++11)
```C++
template<typename T, typename ... Ts>
auto sum(T arg, Ts ... args)
-> typename std::common_type<T, Ts...>::type
  { return arg + sum(args...); }
```
Function overload for **more than one argument** (C++14)
```C++
template<typename T, typename ... Ts>
  auto sum(T arg, Ts ... args)
  { return arg + sum(args...); }
```

## <center>C++ standard containers</center>
**Aim:** provide a set of universal container classes that
 - can **store arbitrary types** (in general, only objects of the same type (in each container; `std::tuple`  for multi-type containers)
 - provide a **uniform interface** to insert, delete, access, and manipulate, items and iterate over the items stored
 - provide optimal implementations of **standard data structures**, e.g., double-linked lists, balanced trees (red-black tree)

## <center>C++ standard containers</center>
• `std::array`:array with compile-time size (non-resizable) 

• `std::vector`:array with run-time size (resizable)

• `std::list`:double-linked list

• `std::forward_list`:single-linked list

• `std::stack`:Last-In-First-Out stack

• `std::queue`:First-In-First-Out queue

• `std::set/std::multiset`:Set of unique elements

• `std::map/std::multimap`:Set of (key,value) elements

## <center>C++ standard containers</center>
Container classes support the following base functionality
   - `size()`: returns the size of the container
   - `empty()`: returns true of the container is empty
   - `swap(container& other)`: swaps contents of containers
Many container classes provide so-called **iterators**
   - `begin()`, `end()`: editable iterator
   - `cbegin()`, `cend()`: constant, i.e., non-editable iterator


## <center>Simple array example</center>
```C++
#include <array>
std::array<int, 5> a = {1, 2, 3, 4, 5};
std::cout << "empty: " << a.empty() << ”\n”; 
std::cout << "size: " << (int) a.size() << ”\n”; 
std::cout << "max_size:" << (int) a.max_size() << “\n”;
for (auto i = 0; i < a.size(); i++)
    std::cout << a[i] << “\n”;
```

## <center>Simple array example</center>
```C++
#include <array>
std::array<int, 5> a = {1, 2, 3, 4, 5};
std::array<int, 4> b = {6, 7, 8, 9};

a.swap(b);

std::cout << "size: " << (int) a.size() << ”\n”;
std::cout << "size: " << (int) b.size() << “\n”;
```

## <center>Simple vector example</center>
```C++
#include <vector>
  std::vector<int> v;
  v.reserve(20);
  v.push_back(42);
  v.push_back(11);
  v.push_back(1);
...
  for (auto i = 0; i < v.size(); ++i)
      std::cout << v[i] << “\n”;
```